# Semi-automatization of DRN and SFR sensitivity analysis/calibration

**Worflow**\
DRN
1. Load .drn file - *completed*
2. Change the conductance of the cells - *completed*
3. Write the .drn file - *completed*
4. Run the model
5. Read .cbb file
6. Store the outflow data of the first X cells
7. Put the process in a loop over a range of parameters

SFR
1. Load .sfr file
2. Change the hydraulic conductivity of the cells
3. Write the .sfr file
4. Run the model
5. Read streamflow.dat file
6. Store the flux data of cell at reach X
7. Put the process in a loop over a range of parameters

## Setup

In [19]:
import flopy
import flopy.utils.binaryfile as bf
import glob
import numpy as np
import os
import pandas as pd

#check if the below lines are still useful at the end
import sys
from pprint import pformat
from tempfile import TemporaryDirectory

In [2]:
# cwd = "C:/Users/user/OneDrive - Politecnico di Milano/hydrogeo-modelling/TEAM_Idrogeo/Tesi/Tesi_Ceola_Pirovano/Tesi_Pirovano/busca_drain_start"
cwd = os.getcwd()

In [34]:
def import_input_file(path):
    f = open(path).readlines()
    df = pd.DataFrame()
    for row in f[4:]:
        r = list(filter(None, row.split(' ')))
        df = pd.concat([df, pd.DataFrame(r).transpose()])
    df.columns = ['layer', 'row', 'column', 'stage', 'conductance', 'node']
    df.node = df.node.str.removesuffix('\n')
    df.layer = df.layer.astype('int')
    df.row = df.row.astype('int')
    df.column = df.column.astype('int')
    df.stage = df.stage.astype('float')
    df.conductance = df.conductance.astype('float')
    df.reset_index(inplace=True, drop = True)
    return df

## DRAIN

In [35]:
# Load .drn file
drn = import_input_file(os.path.join(cwd, 'test_files', 'busca_drain.drn'))
drn.head()

,layer,row,column,stage,conductance,node
0,1,46,51,124.838,0.012,0
1,1,47,51,124.835,0.018,0
2,1,48,51,124.832,0.006,0
3,1,48,52,124.830,0.024,0
4,1,49,52,124.824,0.018,0


In [5]:
# Load DRAIN cells characteristics
drn_sp = pd.read_csv(os.path.join(cwd, 'test_files', 'busca_drain_specifiche_celle.csv'))

In [6]:
# Compute the conductance of each cell for a given hydraulic conductivity
# Change the conductance column
k = 0.0001 #hydraulic conductivity
conductance_change = (k*drn_sp.Width*drn_sp.Length)/drn_sp.Thickness
drn.conductance = conductance_change

### Test: write .drn file using flopy

In [42]:
modelpth = os.path.join(cwd, 'test_files')

# creat the model package
mf = flopy.modflow.Modflow(
    "drn_test",
    model_ws=modelpth,
    exe_name="mf2005",
)
ipakcb = 50 #code for cell-by-cell flow data storage

c:\Users\user\anaconda3\envs\fontanili\Lib\site-packages\flopy\mbase.py:97: UserWarning: The program mf2005 does not exist or is not executable.
  warn(


In [43]:
# transform the drn structure in the flopy required format for "stress_period_data"
drn.layer = 0
stress_period_data = {0: drn.iloc[:, :-1].to_numpy().tolist()} #remove node column, not needed nor supported by flopy's ModflowDrn class

In [44]:
drain = flopy.modflow.ModflowDrn(mf, ipakcb=ipakcb, stress_period_data=stress_period_data,
                                 filenames=os.path.join(cwd, 'test_files', 'drn_input_file_test_v4.drn'))
drain.write_file(check = False)

In [10]:
# bug found:
# the layer gets put equal to 2 instead of 1
# the problem is not in the stress_period_data table passed to drn
# check stress_period_data.write_transient method
# couldn't figure it out, so found a way out: set drn.layer = 0
# drn_input_file_test_v4.drn works

### Test: write .drn file autonomously without using flopy

Riprovare con il pacchetto di flopy usando un numpy array invecedel dataframe


Da qui stavo scrivendo write_file per scrivermi i dati
https://flopy.readthedocs.io/en/latest/_modules/flopy/modflow/mfdrn.html#ModflowDrn.write_file

stavo indagando come flopy gestisce lo stress_period_data (MFils o qualcosa)

In [11]:
ipakcb = 1
line = f"{2:10d}{ipakcb:10d}"
line

'         2         1'

In [38]:
heading = '# DRN package for MODFLOW-2005 generated by paolchol'
n_drain_cells = drn.shape[0]
ipakcb = 50


with open(os.path.join(cwd, 'drn_input_file_test.drn'), 'w') as file:
    file.write(f"{heading}\n")
    line = f"{n_drain_cells:10d}{ipakcb:10d}"
    for opt in self.options:
        line += " " + str(opt)
    line += "\n"
    file.write(line)

    # for row in drn.iterrows():
    #     file.writelines(row)

,index,layer,row,column,stage,conductance,node
0,0,1,46,51,124.838,0.0012,0
1,0,1,47,51,124.835,0.0018,0
2,0,1,48,51,124.832,0.0006,0
3,0,1,48,52,124.830,0.0024,0
4,0,1,49,52,124.824,0.0018,0
...,...,...,...,...,...,...,...
323,0,1,264,156,124.041,0.0012,0
324,0,1,265,156,124.035,0.0018,0
325,0,1,266,156,124.027,0.0018,0
326,0,1,266,157,124.023,0.0006,0


## Read .cbb output file

class CellBudgetFile\
https://flopy.readthedocs.io/en/latest/source/flopy.utils.binaryfile.html

The outflow will have to be considered until cell:
- row 92, column 76 (last cell of first segment)
Check if any substantial change in outflow happen if considering:
- row 93, column 76 (first cell of second segment)

In [ ]:
cbb = bf.CellBudgetFile(Path(workspace) / f"{name}.cbc")

## SFR

### Store the flow at reach x

Consider the outward flow of these two cells:
- reach 72, segment 1
- reach 1, segment 2